In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np
import pickle

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model = Sequential()
    model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.2))

    model.add(Dense(1, activation="sigmoid"))

    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,
                  metrics=["acc"])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=8,
              epochs=5,
              verbose=1)

    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
154/154 [==============================] - 9s 44ms/step - loss: 0.0000e+00 - acc: 0.5521
Epoch 2/5
154/154 [==============================] - 7s 44ms/step - loss: 0.0000e+00 - acc: 0.5736
Epoch 3/5
154/154 [==============================] - 7s 45ms/step - loss: 0.0000e+00 - acc: 0.5620
Epoch 4/5
154/154 [==============================] - 7s 44ms/step - loss: 0.0000e+00 - acc: 0.5988
Epoch 5/5
154/154 [==============================] - 7s 44ms/step - loss: 0.0000e+00 - acc: 0.5965
Score for fold 1: loss of 0.0; acc of 62.012988328933716%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
154/154 [==============================] - 7s 44ms/step - loss: 0.0000e+00 - acc: 0.5893
Epoch 2/5
154/154 [==============================] - 7s 45ms/step - loss: 0.0000e+00 - acc: 0.5897
Epoch 3/5
 72/154 [=============>................] - ET

KeyboardInterrupt: 

In [ ]:
# Generate generalization metrics
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['loss'])
plt.title('loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['acc'])
plt.title('accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()